In [1]:
from random import randint, choice  #<-- used during random ip address generation
import requests as rq               #<-- for api requests
import json, os, csv                #<-- For file operations
from tqdm import tqdm               #<-- Progressbar module

In [2]:
class IP_Generator:
    # This Class can be replaced by Faker module but i will keep this, why not!
    @staticmethod
    def generate_public_IP(count=1):
        
        #following list is for all possible ranges of public IPs in class A, B and C
        public_ip_ranges = [[(1, 9), (0, 255), (0, 255), (0, 255)],
                            [(11, 126), (255, 255), (255, 255), (255, 255)],
                            [(128, 168), (0, 255), (0, 255), (0, 255)],
                            [(169, 169), (0, 253), (0, 255), (0, 255)],
                            [(169, 169), (255, 255), (0, 255), (0, 255)],
                            [(170, 171), (0, 255), (0, 255), (0, 255)],
                            [(172, 172), (0, 15), (0, 255), (0, 255)],
                            [(172, 172), (32, 255), (0, 255), (0, 255)],
                            [(173, 191), (0, 255), (0, 255), (0, 255)],
                            [(192, 192), (0, 167), (0, 255), (0, 255)],
                            [(193, 223), (0, 255), (0, 255), (0, 255)]
                           ]
        
        #intiating IP_set to get all unique IPs [Just in case!]
        IP_set = set()
        while len(IP_set) < count:
            ch = choice(public_ip_ranges)
            ip = ".".join([str(randint(*ch[0])),
                        str(randint(*ch[1])),
                        str(randint(*ch[2])),
                        str(randint(*ch[3])) ])

            IP_set.add(ip)

        return list(IP_set)

    @staticmethod
    def generate_any_IP(count=1):
        IP_set = set()
        while len(IP_set) < count:
            ip = ".".join([str(randint(1, 255)),
                        str(randint(0, 255)),
                        str(randint(0, 255)),
                        str(randint(0, 255)) ])

            IP_set.add(ip)

        return list(IP_set)

In [3]:
class IP_locator:
    def __init__(self, IP_list, *args):
        self.IP_list = IP_list
        self.fields = args

    def get_location(self):
        location_info = []
        
        access_key = "<Your API Key here>"
        request_url = "http://api.ipstack.com/{}?access_key="+access_key
        
        if len(self.fields):
            request_url += "&fields=" + ",".join(self.fields)

        for ip in tqdm(self.IP_list):
            data = json.loads(rq.get(request_url.format(ip)).text)
            location_info.append(data)

        return location_info

In [4]:
class Logger:
    def __init__(self, file_name):
        self.file_name = file_name

    def log_csv(self, data_list):
        extension = ".csv"
        try:
            if not os.path.exists(self.file_name+extension):
                with open(self.file_name + extension, "a+") as file:
                    fields = []
                    fields.extend(data_list[0].keys())

                    csv_file = csv.writer(file, delimiter=",")
                    csv_file.writerow(fields)

            with open(self.file_name + extension, "a+") as file:
                csv_file = csv.writer(file, delimiter=",")

                for i in tqdm(data_list):
                    row = []
                    row.extend(i.values())
                    csv_file.writerow(row)
            return True

        except Exception as e:
            print(e)
            return False

    def log_txt(self, data_list):
        extension = ".txt"
        try:
            with open(self.file_name + extension, "a+") as txt_file:
                for record in tqdm(data_list):
                    data_line = json.dumps(record) + "\n\n"
                    txt_file.write(data_line)
            return True

        except Exception as e:
            return False


    def read_txt_data(self):
        extension = ".txt"
        try:
            with open(self.file_name + extension, "r") as txt_file:
                file_data = txt_file.read()
                
                if len(file_data) == 0:
                    return 0
                
                string_records = file_data.split("\n\n")[:-1]
                records_list = []
                for i in string_records:
                    records_list.append(json.loads(i))
                return records_list
        except Exception as e:
#             print(e)
            return False

In [5]:
def main():
    count = int(input("Enter count of IPs to LOG :"))
    IP_list = IP_Generator.generate_public_IP(count=count)
    
    print("\nGetting Geolocations of "+str(count)+" IP(s)...\n")

    locator = IP_locator(IP_list, "ip", "city", "region_name", "country_name")
    location_data = locator.get_location()
    

    print("\n\nLogging data to file...")

    logger_obj = Logger("IP_locations")
#     if logger_obj.log_csv(location_data):
#         print("CSV File Done!")
#     else :
#         print("Some Problem Occured while Logging CSV File!")
    if logger_obj.log_txt(location_data):
        print("Text File Done!")
    else :
        print("Some Problem Occured while Logging Txt File!")
        
    file_data = logger_obj.read_txt_data()
    if file_data:
        print("\n" + "-"*15 + " Text File Data " + "-"*15 + "\n")
        for record in file_data:
            print("  IP Address : " + record['ip'])
            print("\tCity :" + record['city'])
            print("\tRegion : " + record['region_name'])
            print("\tCountry : " + record['country_name'] + "\n")

In [6]:
main()

Enter count of IPs to LOG :200

Getting Geolocations of 200 IP(s)...



100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [02:11<00:00,  1.52it/s]




Logging data to file...


100%|█████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 27722.69it/s]


Text File Done!

--------------- Text File Data ---------------

  IP Address : 181.7.163.34
	City :Buenos Aires
	Region : Buenos Aires F.D.
	Country : Argentina

  IP Address : 192.82.63.85
	City :Putrajaya
	Region : Putrajaya
	Country : Malaysia

  IP Address : 1.40.1.42
	City :Parramatta
	Region : New South Wales
	Country : Australia

  IP Address : 172.217.159.122
	City :Mountain View
	Region : California
	Country : United States

  IP Address : 169.253.186.237
	City :Shaw
	Region : Washington
	Country : United States

  IP Address : 169.255.80.161
	City :Gaborone
	Region : Gaborone
	Country : Botswana

  IP Address : 6.237.195.66


TypeError: can only concatenate str (not "NoneType") to str